# Running LightGBM Distributed using Generic Estimator and HyperDrive

### 1. Get AML Objects

Specifically retrieve these objects:
1. Workspace `ws` 
1. Environment `lgbm_env` 
1. Compute Target `training_cluster` 
1. Validation Dataset `valid_ds` 
1. Training Dataset `train_ds` 

In [ ]:
from azureml.core import Workspace, Experiment, Dataset, Environment
from azureml.train.estimator import Estimator, Mpi

ws = Workspace.from_config()

In [ ]:
ENVIRONMENT_NAME = 'lightgbm-cli'
lgbm_env = Environment.get(ws, ENVIRONMENT_NAME)

In [ ]:
training_cluster = ws.compute_targets['training-cluster']
valid_ds = ws.datasets['renewal_test_csv']
train_ds = ws.datasets['renewal_train_csv']

In [ ]:
EXPERIMENT_NAME = 'lightgbm-parallel'
HYPERDRIVE_RUN = False

### 2. Creating Estimator object
The estimator will run [`train.py`](../code/train.py) script. `train.py` will accept [any parameters that LightGBM accepts.](https://lightgbm.readthedocs.io/en/latest/Parameters.html) These will be passed to the command line tool and (if MPI is enabled) will be run in parallel.

In [ ]:
# Specify the number of nodes to run on
n_nodes = 1

LABEL_COLUMN_NAME = 'i_year1_renewal_flag'

static_parameters = {
    '--train_data': train_ds.as_named_input('train').as_mount(),
    '--valid_data': valid_ds.as_named_input('valid').as_mount(),
    '--task': 'train',
    '--conf_file': 'train.conf',
    '--metric': 'auc,binary_logloss,binary_error,mean_absolute_error',
    '--num_machines': n_nodes,
    '--label_column': f"name:{LABEL_COLUMN_NAME}",
    '--num_iterations': 100,
    '--tree-learner': 'voting'
}

lgbm_est = Estimator(source_directory='../code/',
                     entry_script='train.py', 
                     script_params=static_parameters,
                     compute_target=training_cluster,
                     environment_definition=lgbm_env,
                     distributed_training=None if n_nodes == 1 else Mpi(),
                     node_count=n_nodes
                     )

### 3. Specifying Hyperdrive Settings
As with the previous step, the train.py script will accept [any parameters that LightGBM accepts.](https://lightgbm.readthedocs.io/en/latest/Parameters.html) These will be passed to the command line tool and run in parallel.

Here, some sample parameters that could be passed are shown. 

In [ ]:
from azureml.train.hyperdrive import HyperDriveConfig, choice, uniform, randint, RandomParameterSampling, BanditPolicy, PrimaryMetricGoal

hyperdrive_params = RandomParameterSampling({
    '--learning_rate': uniform(0.01, 2),
    '--max_depth': choice(0,200,300,400),
    '--boosting': choice('gbdt', 'rf', 'dart'),
    '--bagging_freq': randint(10),
    '--lambda_l1': uniform(0.01, 2),
    '--lambda_l2':uniform(0.01, 2)
})

etp = BanditPolicy(evaluation_interval=5, slack_factor=0.1, delay_evaluation=10)

hd_runconfig = HyperDriveConfig(estimator=lgbm_est, 
                                   hyperparameter_sampling=hyperdrive_params, 
                                   primary_metric_name='validation_auc', 
                                   primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                                   max_total_runs=20,
                                   policy=etp,
                                   max_concurrent_runs=2)

In [ ]:
experiment = Experiment(ws, EXPERIMENT_NAME)
run = experiment.submit(hd_runconfig if HYPERDRIVE_RUN else lgbm_est)

In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()

In [ ]:
run.wait_for_completion(show_output=True)

<br><br><br><br><br><br>
###### Copyright (c) 2020 Microsoft

###### This software is released under the MIT License.
###### https://opensource.org/licenses/MIT